<a href="https://colab.research.google.com/github/ampnb/Mind/blob/main/MW_Recording.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using Display!**

In [ ]:
import time
import mne
import os
import tkinter as tk

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowPresets

def display_instructions(root, label, text):
    label.config(text=text)
    root.update()

def run_condition_instructions(root, label, condition):
    instructions = f"Now, {condition} for 5 minutes.\nPress ENTER when you are ready."
    display_instructions(root, label, instructions)
    root.bind('<Return>', lambda event: root.quit())
    root.mainloop()

def main():
    BOARD_ID = 8
    PARTICIPANT_ID = "Participant1"
    RECORDING_DIR = "./"
    n_trials = 4
    trial_duration = 5 * 60

    # Setup the graphical display
    root = tk.Tk()
    root.attributes("-fullscreen", True)

    label = tk.Label(root, text="", font=("Helvetica", 24), wraplength=root.winfo_screenwidth())
    label.pack(pady=(root.winfo_screenheight() // 2) - 100)

    BoardShim.enable_dev_board_logger()
    params = BrainFlowInputParams()
    board = BoardShim(BOARD_ID, params)
    board.prepare_session()
    board.start_stream()

    # Resting state
    display_instructions(root, label, "Resting state recording in progress. Keep your eyes closed and relax.")
    time.sleep(5 * 60)  # 5 minutes for baseline recording

    for trial in range(n_trials):
        # Meditation condition
        run_condition_instructions(root, label, "focus on your breath or another object of attention (meditation condition)")
        data = record_eeg_data(board, trial_duration)
        raw = getdata(data, BOARD_ID)
        save_raw(raw, RECORDING_DIR, f"{PARTICIPANT_ID}_meditation_trial{trial + 1}")

        # Mind-wandering condition
        run_condition_instructions(root, label, "let your thoughts wander freely without focusing on anything specific (mind-wandering condition)")
        data = record_eeg_data(board, trial_duration)
        raw = getdata(data, BOARD_ID)
        save_raw(raw, RECORDING_DIR, f"{PARTICIPANT_ID}_mindwandering_trial{trial + 1}")

    board.stop_stream()
    board.release_session()

    # Close the tkinter window
    root.destroy()

def record_eeg_data(board, duration):
    time.sleep(duration)
    data = board.get_board_data()
    return data

# ... (The rest of the code remains the same) ...

if __name__ == "__main__":
    main()

# **Not Use Display!**

In [ ]:
import time
import mne
import os

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowPresets

def run_condition_instructions(condition):
    print(f"Now, {condition} for 5 minutes.")
    print("Press ENTER when you are ready.")
    input()

def main():
    BOARD_ID = 8
    PARTICIPANT_ID = "Participant1"
    RECORDING_DIR = "./"
    n_trials = 4
    trial_duration = 5 * 60

    BoardShim.enable_dev_board_logger()
    params = BrainFlowInputParams()
    board = BoardShim(BOARD_ID, params)
    board.prepare_session()
    board.start_stream()

    # Resting state
    print("Resting state recording in progress. Keep your eyes closed and relax.")
    time.sleep(5 * 60)  # 5 minutes for baseline recording

    for trial in range(n_trials):
        # Meditation condition
        run_condition_instructions("focus on your breath or another object of attention (meditation condition)")
        data = record_eeg_data(board, trial_duration)
        raw = getdata(data, BOARD_ID)
        save_raw(raw, RECORDING_DIR, f"{PARTICIPANT_ID}_meditation_trial{trial + 1}")

        # Mind-wandering condition
        run_condition_instructions("let your thoughts wander freely without focusing on anything specific (mind-wandering condition)")
        data = record_eeg_data(board, trial_duration)
        raw = getdata(data, BOARD_ID)
        save_raw(raw, RECORDING_DIR, f"{PARTICIPANT_ID}_mindwandering_trial{trial + 1}")

    board.stop_stream()
    board.release_session()

def record_eeg_data(board, duration):
    time.sleep(duration)
    data = board.get_board_data()
    return data

# ... (The rest of the code remains the same) ...

if __name__ == "__main__":
    main()

# Using PsychoPy

In [ ]:
import time
import mne
import os
import tkinter as tk
from mne.datasets import eegbci
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowPresets
# from psychopy import core, event, visual 

def display_instructions(root, label, text):
    label.config(text=text)
    root.update()

def run_condition_instructions(root, label, condition):
    instructions = f"Now, {condition} for 5 minutes.\nPress ENTER when you are ready."
    display_instructions(root, label, instructions)
    root.bind("<Return>", lambda event: root.quit())
    root.mainloop()

def getdata(data,board):

    eeg_channels = BoardShim.get_eeg_channels(board)
    data[eeg_channels] = data[eeg_channels] / 1e6


    data = data[eeg_channels]
    ch_names = BoardShim.get_eeg_names(board)
    ch_types = (['eeg'] * len(eeg_channels))

    sfreq = BoardShim.get_sampling_rate(board)
    
    #Create Raw data from MNE
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
    raw = mne.io.RawArray(data, info)
    # print(raw)
    raw_data = raw.copy()
    eegbci.standardize(raw_data)
    montage = mne.channels.make_standard_montage('standard_1020')
    raw_data.set_montage(montage)
    
    return raw_data

def create_session_folder(subj,dir):
    base_path = os.getcwd() + "\\"
    dir = base_path + dir
    folder_name = f'{subj}'
    print(folder_name)
    if os.path.isdir(os.path.join(dir, folder_name)):
        folder_path = os.path.join(dir, folder_name)
    else:
        folder_path = os.path.join(dir, folder_name)
        os.makedirs(folder_path)
    return folder_path

def save_raw(raw, dir, participant_id):
    folder_path = create_session_folder(participant_id,dir)
    raw.save(os.path.join(folder_path, f'{participant_id}{".fif"}'), overwrite = True)
    return os.path.basename(folder_path)

def main():
    BOARD_ID = 8
    PARTICIPANT_ID = "tan"
    RECORDING_DIR = "./"
    n_trials = 4
    trial_duration = 5 * 60
    baseline_duration = 5 * 60
    # trial_duration = 15
    # baseline_duration = 15

    # Setup the graphical display
    root = tk.Tk()
    root.attributes("-fullscreen", True)

    label = tk.Label(root, text="", font=("Helvetica", 24), wraplength=root.winfo_screenwidth())
    label.pack(pady=(root.winfo_screenheight() // 2) - 100)

    BoardShim.enable_dev_board_logger()
    params = BrainFlowInputParams()
    board = BoardShim(BOARD_ID, params)
    board.prepare_session()
    board.start_stream()

    # Resting state
    display_instructions(root, label, "Resting state recording in progress. Keep your eyes closed and relax.")
    data = record_eeg_data(board, baseline_duration)
    raw = getdata(data, BOARD_ID)
    save_raw(raw, RECORDING_DIR, f"{PARTICIPANT_ID}_resting_state")

    for trial in range(n_trials):
        # Meditation condition
        run_condition_instructions(root, label, "focus on your breath or another object of attention (meditation condition)")
        data = record_eeg_data(board, trial_duration)
        raw = getdata(data, BOARD_ID)
        save_raw(raw, RECORDING_DIR, f"{PARTICIPANT_ID}_meditation_trial{trial + 1}")

        # Mind-wandering condition
        run_condition_instructions(root, label, "let your thoughts wander freely without focusing on anything specific (mind-wandering condition)")
        data = record_eeg_data(board, trial_duration)
        raw = getdata(data, BOARD_ID)
        save_raw(raw, RECORDING_DIR, f"{PARTICIPANT_ID}_mindwandering_trial{trial + 1}")

    board.stop_stream()
    board.release_session()

    # Close the tkinter window
    root.destroy()

def record_eeg_data(board, duration):
    time.sleep(duration)
    data = board.get_board_data()
    return data

# ... (The rest of the code remains the same) ...

if __name__ == "__main__":
    main()